<div class = "row">
    <div class = "colums">
        <img src="..\trecslogo.png" align="left" alt="Drawing" width ="60"/>    
    </div>
    <div class = "colums">
        <img src="..\asrlogo.png" align="right" alt="Drawing" width ="175"/>
    </div>    
</div>

# SWAPTION

## Algemeen

### Import en instellingen

In [1]:
from classSwaption import *
import pandas as pd
import numpy as np
from datetime import datetime
from bokeh.plotting import figure, output_file, ColumnDataSource, output_notebook
from bokeh.models import HoverTool, NumeralTickFormatter, FactorRange
from bokeh.io import show
output_notebook(hide_banner=True)

Links uitlijnen tabellen

In [2]:
%%html
<style>
    table {
        display: inline-block
    }
</style>

Bij printen van een dataframe wordt slechts een beperkt aantal rijen getoond.

In [3]:
pd.set_option('display.max_rows', 300)

## Initialisatie parameters

In [4]:
# data
datumKwartaal = 20220331
datumActueel = 20220331 # in dit notebook veronderstel ik dat de actuele datum dezelfde is als de kwartaal datum
data = (datumKwartaal, datumActueel)

# naam van de basis curve
renteCurveBasis = 'FairValue'

## Importeer en bewerk data

### Rentecurves per kwartaal en actueel

In [5]:
df_curves = pd.read_excel(r"curvebestanden/curvesAssets.xlsx", decimal = '.')

df_curves['datum'] = datumKwartaal # in dit notebook dezelfde als actuele datum
datum_col = df_curves.pop('datum')
df_curves.insert(0, 'datum', datum_col)

pd.options.display.float_format = '{:,.8f}'.format
df_curves
#df_curves.to_excel('output.xlsx')

,datum,Jaar,Currency,FairValue,swap.cra.zero.va.down,swap.cra.zero.va.up,SII_basis,SII_Yield_Curve_down,SII_Yield_Curve_up,SII_basis.EQUITY_TYPE_1,...,swap.cra.eur-stylized-1.zero.va.sw345.down345,swap.cra.eur-stylized-1.zero.va.sw345.up345,swap.cra.dnb,swap.cra.dnb.up,swap.cra.dnb.down,swap.cra.zero.va-ratio50.sw270,swap.cra.zero.va-ratio50.sw270.down270,swap.cra.zero.va-ratio50.sw270.up270,swap.cra.zero.va-ratio50.sw270_Currency_Up,swap.cra.zero.va-ratio50.sw270_Currency_Down
0,20220331,1,EUR,-0.00080869,-0.00080869,0.00919131,-0.00080869,-0.00080869,0.00919131,-0.00080869,...,-0.00080776,0.00919224,-0.00080869,0.00919131,-0.00080869,-0.00080869,-0.00080869,0.00919131,-0.00080869,-0.00080869
1,20220331,2,EUR,0.00537754,0.00253036,0.01537754,0.00537754,0.00254137,0.01537754,0.00537754,...,0.00254794,0.01538412,0.00537754,0.01537754,0.00253036,0.00537754,0.00254137,0.01537754,0.00537754,0.00537754
2,20220331,3,EUR,0.00806788,0.00411486,0.01806788,0.00806788,0.00412935,0.01806788,0.00806788,...,0.00413130,0.01806983,0.00806788,0.01806788,0.00411486,0.00806788,0.00412935,0.01806788,0.00806788,0.00806788
3,20220331,4,EUR,0.00926595,0.00510382,0.01926595,0.00926595,0.00511846,0.01926595,0.00926595,...,0.00511910,0.01926659,0.00926595,0.01926595,0.00510382,0.00926595,0.00511846,0.01926595,0.00926595,0.00926595
4,20220331,5,EUR,0.00990248,0.00577112,0.01990248,0.00990248,0.00578518,0.01990248,0.00990248,...,0.00578534,0.01990264,0.00990248,0.01990248,0.00577112,0.00990248,0.00578518,0.01990248,0.00990248,0.00990248
5,20220331,6,EUR,0.01037414,0.00640208,0.02037414,0.01037414,0.00641534,0.02037414,0.01037414,...,0.00641525,0.02037405,0.01037414,0.02037414,0.00640208,0.01037414,0.00641534,0.02037414,0.01037414,0.01037414
6,20220331,7,EUR,0.01078707,0.00693993,0.02078707,0.01078707,0.00695264,0.02078707,0.01078707,...,0.00695239,0.02078682,0.01078707,0.02078707,0.00693993,0.01078707,0.00695264,0.02078707,0.01078707,0.01078707
7,20220331,8,EUR,0.01120210,0.00749941,0.02120210,0.01120210,0.00751155,0.02120210,0.01120210,...,0.00751119,0.02120174,0.01120210,0.02120210,0.00749941,0.01120210,0.00751155,0.02120210,0.01120210,0.01120210
8,20220331,9,EUR,0.01161711,0.00808305,0.02161711,0.01161711,0.00809461,0.02161711,0.01161711,...,0.00809416,0.02161665,0.01161711,0.02161711,0.00808305,0.01161711,0.00809461,0.02161711,0.01161711,0.01161711
9,20220331,10,EUR,0.01205979,0.00861073,0.02205979,0.01205979,0.00862207,0.02205979,0.01205979,...,0.00862154,0.02205925,0.01205979,0.02205979,0.00861073,0.01205979,0.00862207,0.02205979,0.01205979,0.01205979


In [6]:
curves = list(df_curves.columns)[3:]
curves

['FairValue',
 'swap.cra.zero.va.down',
 'swap.cra.zero.va.up',
 'SII_basis',
 'SII_Yield_Curve_down',
 'SII_Yield_Curve_up',
 'SII_basis.EQUITY_TYPE_1',
 'SII_basis.EQUITY_TYPE_2',
 'SII_basis.EQUITY_QUINF',
 'SII_basis.EQUITY_QUINFC',
 'SII_basis.EQUITY_Average',
 'SCR_Average_zonder_OMA',
 'SCR_Average_OMA_Q2',
 'SCR_Average_OMA_Q4',
 'SII_Real_Estate',
 'SCR_Currency_Up',
 'SCR_Currency_Down',
 'ECAP_basis',
 'ECAP_Yield_Curve_down',
 'ECAP_Yield_Curve_up',
 'ECAP_basis.EQUITY_VM_EURO',
 'ECAP_basis.EQUITY_Deelnemingen',
 'ECAP_basis.EQUITY_Alternatives',
 'ECAP_basis.EQUITY_Emerging_Markets',
 'ECAP_basis.EQUITY_VM_NON_EURO',
 'ECAP_Real_Estate',
 'ECAP_Currency_Up',
 'ECAP_Currency_Down',
 'SII_Real_Estate_-10procent',
 'Vastgoed_-10procent',
 'Aandelen_-20procent.EQUITY_TYPE_1',
 'Aandelen_-20procent.EQUITY_TYPE_2',
 'Aandelen_-20procent.EQUITY_QUINF',
 'Aandelen_-20procent.EQUITY_QUINFC',
 'Aandelen_-20procent.EQUITY_Average',
 'Aandelen_-20procent',
 'swap.cra.zero.va.plus20bp

### IMW bestand

CIC codes swaptions:

- B6 = Call options granting its owner the right but not the obligation to enter into a long position in an underlying swap, i.e., enter into a swap where the owner pays the fixed leg and receive the floating leg
- C6 = Put options granting its owner the right but not the obligation to enter into a short position in an underlying swap, i.e., enter into a swap in which the owner will receive the fixed leg, and pay the floating leg

In [7]:
col_list = ['Reporting Date', 'Cic Id Ll', 'Market Value EUR LL', 'BalNomVal LT', 'Strike Price Laagste Level', 'Maturity Call LL', 'Maturity LL', 'Security Id Ll', 'Security Type Ll', 'Volatility dim']
df_swaption = pd.read_csv(r"imwbestand/2022M03 Adjusted IMW Weekupdate 202205131221.csv", usecols = col_list, sep = ";", decimal = '.', encoding= 'unicode_escape', low_memory=False)

In [8]:
df_swaption = df_swaption.loc[df_swaption['Security Type Ll'] == 'SWAPTION']
df_swaption = df_swaption.reset_index(drop=True)

In [9]:
df_swaption['dateActual'] = pd.to_datetime(df_swaption['Reporting Date'], format='%Y%m%d').dt.date
df_swaption['dateSwapExpiry'] = pd.to_datetime(df_swaption['Maturity LL'], format='%Y%m%d').dt.date
df_swaption['dateSwaptionExpiry'] = pd.to_datetime(df_swaption['Maturity Call LL'], format='%Y%m%d').dt.date
df_swaption = df_swaption.drop(['Security Type Ll', 'Reporting Date', 'Maturity LL', 'Maturity Call LL'], axis = 1)

In [10]:
df_swaption.rename(columns = { \
                              'Cic Id Ll':'cicCode', \
                              'Security Id Ll':'securityId', \
                              'Market Value EUR LL':'marketValue', \
                              'BalNomVal LT':'nominalValue', \
                              'Volatility dim':'volatilityPercentageIMW', \
                              'Strike Price Laagste Level':'strikePercentage' \
                             }, inplace = True)
pd.options.display.float_format = '{:,.2f}'.format
df_swaption

,cicCode,securityId,marketValue,nominalValue,volatilityPercentageIMW,strikePercentage,dateActual,dateSwapExpiry,dateSwaptionExpiry
0,XLB6,3350972,"2,332,319.63","50,000,000.00",0.58,2.75,2022-03-31,2068-06-03,2038-06-01
1,XLC6,3350574,"965,615.72","2,000,000.00",1.43,4.00,2022-03-31,2042-09-21,2022-09-19
2,XLC6,3350415,"1,454,048.72","3,000,000.00",1.54,4.00,2022-03-31,2042-07-28,2022-07-26
3,XLB6,3350971,"3,485,393.65","50,000,000.00",0.56,2.10,2022-03-31,2068-06-03,2038-06-01
4,XLB6,3350980,"1,790,950.35","35,000,000.00",0.57,2.60,2022-03-31,2068-07-29,2038-07-27
5,XLB6,3350966,"3,657,941.28","100,000,000.00",0.60,3.00,2022-03-31,2063-05-06,2038-05-04
6,XLB6,3350962,"9,137,107.18","140,000,000.00",0.57,2.20,2022-03-31,2067-09-07,2037-09-03
7,XLB6,SWOP3351002,"4,558,959.82","25,000,000.00",0.50,0.50,2022-03-31,2070-04-04,2040-03-29
8,XLB6,SWOP3350996,"4,560,215.34","25,000,000.00",0.50,0.50,2022-03-31,2070-03-28,2040-03-26
9,XLB6,SWOP3351020,"6,735,160.21","50,000,000.00",0.50,1.00,2022-03-31,2071-02-13,2041-02-11


## Aanmaken van alle swaption objecten

### Initiëren van alle swaption objecten

In [11]:
swaptions = []
for index, row in df_swaption.iterrows():
    swaption = Swaption(row['cicCode'], row['securityId'], row['marketValue'], row['nominalValue'], row['volatilityPercentageIMW'], row['strikePercentage'], row['dateSwapExpiry'], row['dateSwaptionExpiry'])
    swaptions.append(swaption)
    

In [12]:
swaptions[3].swaptionType

'call'

### Berekenen van implied volatility voor alle swaption objecten

In [13]:
curveBasis = df_curves[renteCurveBasis].to_list()
print(curveBasis)

[-0.0008086935257475281, 0.0053775440399868035, 0.00806788126333391, 0.009265953621076184, 0.009902479244357831, 0.010374143736326058, 0.010787068455935911, 0.011202102823459237, 0.011617107100261759, 0.012059787141615708, 0.012374994102317238, 0.012641946816562388, 0.012794557383237537, 0.012925384745765545, 0.013038782129757731, 0.012875739488164228, 0.012731900125585227, 0.012604060062879174, 0.012489690527918595, 0.012386768991023756, 0.012100226142645276, 0.011839803022673623, 0.011602083914195038, 0.011384223793654114, 0.01118383392157063, 0.010925505228976728, 0.010686370839304127, 0.010464368128510815, 0.010257719785215613, 0.010064886127397532, 0.009849157306174305, 0.009646953382088874, 0.009457041122749477, 0.009278332804816936, 0.009109865368200465, 0.008950783054096911, 0.008800322866011845, 0.008657802332384223, 0.008522609156661476, 0.008394192423701874, 0.008252453170417118, 0.00811748192773809, 0.007988805244417918, 0.007865992822766898, 0.007748652711565995, 0.0076364

In [15]:
for i in range(len(swaptions)):
    swaptions[i].getImpliedVolatility(curveBasis)

In [16]:
swaptions[4].impliedVolatility

3581900.7

## Berekenen van alle swaption waarden voor alle curves

In [ ]:
columnNames = ['securityId'] + curves
df_swaptionPrices = pd.DataFrame(columns = columnNames)
for i in range(len(swaptions)):
    for curve in curves:
        df_swaptionPrices.loc[i, 'securityId'] =  swaptions[i].securityId
        df_swaptionPrices.loc[i, curve] = swaptions[i].getPrice(curve)
    

In [ ]:
df_swaptionPrices

## Documentatie
In dit document staat de methode voor het berekenen van de swaptions beschreven. 

## Modellering

In [ ]:
def berekenTimingDelta(datumKwartaal, datum):    
    datumKwartaalDate = datetime.strptime(str(datumKwartaal), '%Y%m%d')
    datumDate = datetime.strptime(str(datum), '%Y%m%d')
    timingDelta = (datumDate - datumKwartaalDate).days/365
    return timingDelta

In [ ]:
def berekenForwardRente(t, renteCurve):
    if t == 1:
        forwardRente = renteCurve[0]
    else:
        forwardRente = (1+renteCurve[t-1])**t / (1+renteCurve[t-2])**(t-1) - 1
    return forwardRente

In [ ]:
def berekenForwardRentes(renteCurve):
    forwardRentes = []
    for t in range(0,99):
        forwardRente = berekenForwardRente(t+1, renteCurve)
        forwardRentes += [forwardRente]
    return forwardRentes

In [ ]:
def berekenRentesTiming(valuta, datum, timing, renteCurve, curveType):
    rentes = df_curves[curveType].loc[(df_curves[curveType]['datum'] == datum) & (df_curves[curveType]['Currency'] == valuta), renteCurve].values
    jaren = range(0,100)
    rentesTiming = np.interp(timing, jaren, rentes)
    return rentesTiming

## Output

## Verificatie